In [1]:
import pandas as pd

# Load the CSV file
file_path = "ScanRecords.csv"  # Replace with the actual path to your CSV file
try:
    data = pd.read_csv(file_path)
    print("CSV file loaded successfully!")
    print(data.head())  # Display the first few rows of the DataFrame
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

CSV file loaded successfully!
         Date   Time  Duration PatientType
0  2023-08-01   8.23  0.949176      Type 2
1  2023-08-01   8.49  0.479593      Type 1
2  2023-08-01   9.12  0.496112      Type 2
3  2023-08-01  10.26  0.691947      Type 2
4  2023-08-01  10.64  0.345412      Type 1


In [2]:
# Filter the data for Type 1 patients
type1_data = data[data['PatientType'] == 'Type 1']

# Group by date and count arrivals per day
daily_arrivals = type1_data.groupby('Date').size()
lambda_estimate = daily_arrivals.mean()  # Mean of daily arrivals
print(f"Estimated daily arrival rate (λ): {lambda_estimate:.2f}")

Estimated daily arrival rate (λ): 16.48


In [4]:
# Convert 'Duration' to a numpy array
scan_durations = type1_data['Duration'].values
mu_estimate = scan_durations.mean()  # Mean of scan durations
sigma_estimate = scan_durations.std()  # Standard deviation of scan durations
print(f"Estimated mean (μ): {mu_estimate:.2f}")
print(f"Estimated standard deviation (σ): {sigma_estimate:.2f}")

Estimated mean (μ): 0.43
Estimated standard deviation (σ): 0.10


In [10]:
import numpy as np
from scipy.stats import norm

def bootstrap_parametric(n_bootstraps=1000):
    boot_means = []
    boot_stds = []
    for _ in range(n_bootstraps):
        # Generate bootstrap samples from the fitted normal distribution
        boot_sample = norm.rvs(loc=mu_estimate, scale=sigma_estimate, size=len(scan_durations))
        boot_means.append(boot_sample.mean())
        boot_stds.append(boot_sample.std())
    return np.array(boot_means), np.array(boot_stds)

# Run the bootstrap
n_bootstraps = 1000
boot_means, boot_stds = bootstrap_parametric(n_bootstraps)

# Confidence intervals
mean_ci = (np.percentile(boot_means, 2.5), np.percentile(boot_means, 97.5))
std_ci = (np.percentile(boot_stds, 2.5), np.percentile(boot_stds, 97.5))
print(f"95% CI for mean (μ): {mean_ci}")
print(f"95% CI for standard deviation (σ): {std_ci}")

95% CI for mean (μ): (np.float64(0.4218715866383345), np.float64(0.4423800894831401))
95% CI for standard deviation (σ): (np.float64(0.09033253767634766), np.float64(0.10486509073362885))


In [11]:
# The daily_arrivals mean already serves as the MLE for λ in a Poisson distribution
print(f"MLE for daily arrival rate (λ): {lambda_estimate:.2f}")

MLE for daily arrival rate (λ): 16.48


In [12]:
# Estimate confidence intervals for daily arrival rate
arrival_ci = (np.percentile(daily_arrivals, 2.5), np.percentile(daily_arrivals, 97.5))
print(f"95% CI for daily arrivals: {arrival_ci}")

95% CI for daily arrivals: (np.float64(10.55), np.float64(23.0))


In [13]:
# Print previously calculated CI for scan duration
print(f"95% CI for mean (μ): {mean_ci}")
print(f"95% CI for standard deviation (σ): {std_ci}")

95% CI for mean (μ): (np.float64(0.4218715866383345), np.float64(0.4423800894831401))
95% CI for standard deviation (σ): (np.float64(0.09033253767634766), np.float64(0.10486509073362885))


In [37]:
from scipy.stats import norm
import numpy as np

# Given parameters for Type 1 patients
mean_duration = 0.43  # in hours
std_duration = 0.10   # in hours
confidence_level = 0.99

# Calculate the appropriate timeslot length
z_score = norm.ppf((1 + confidence_level) / 2)  # 99% confidence interval
timeslot_length = mean_duration + z_score * std_duration
print(f"Recommended timeslot length for Type 1 patients (99% CI): {timeslot_length:.2f} hours ({timeslot_length * 60:.2f} minutes)")


Recommended timeslot length for Type 1 patients (99% CI): 0.69 hours (41.25 minutes)
